# 📚 4.4相爱篇-搭建网络

🔲能今天做好的事就不要等到明天。以梦为马，学习趁年华。

Tensorflow提供了高度封装的接口，搭建网络非常便捷，下面介绍三种常用的搭建网络的方法。

## 一、本节目标
        本节将详述概述搭建网络的方法，具体包括顺序模型、函数式模型、子类化模型。

## 二、 顺序模型
通过Keras提供的搭建神经网络类Sequential.可以搭建具有单一输出的神经网络。其使用方式有两种：Sequential内置序列和Sequential外置序列。

### 2.1 Sequential内置序列

In [1]:
#引入库文件
# 引入Tensorflow框架
import tensorflow as tf 
# 引入keras
from tensorflow import keras
# 引入keras层结构
from tensorflow.keras import layers 

In [2]:
model = tf.keras.Sequential([
        # 隐藏层-1
        layers.Dense(10, activation="relu", input_shape=(1,), name="layer1"),
        # 隐藏层-2
        layers.Dense(15, activation="relu", name="layer2"),
        # 输出层
        layers.Dense(5, activation="softmax", name="outputs")
    ])
#model.build(input_shape=(4, 28, 28, 1))
    # 展示网络结构
model.summary()
# 绘制网络流程图
keras.utils.plot_model(model, "line-fit-seq.png", show_shapes=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 10)                20        
_________________________________________________________________
layer2 (Dense)               (None, 15)                165       
_________________________________________________________________
outputs (Dense)              (None, 5)                 80        
Total params: 265
Trainable params: 265
Non-trainable params: 0
_________________________________________________________________
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


### 2.2 Sequential外置序列
在 TensorFlow 中，既可以通过自定义权值的底层实现方式搭建神经网络，也可以直接
调用现成的卷积层类的高层方式快速搭建复杂网络。我们主要以 2D 卷积为例。

In [3]:
# Sequential实例化
model = tf.keras.Sequential()
# 添加隐藏层-1
model.add(layers.Dense(10, activation=tf.nn.relu, input_shape=(1,), name="layer1"))
# 添加隐藏层-2
model.add(layers.Dense(15, activation=tf.nn.relu, name="layer2"))
# 添加输出层
model.add(layers.Dense(5, activation=tf.nn.softmax, name="outputs"))
# 展示网络结构
model.summary()
# 绘制网络流程图
keras.utils.plot_model(model, "line-fit-seq-add.png", show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 10)                20        
_________________________________________________________________
layer2 (Dense)               (None, 15)                165       
_________________________________________________________________
outputs (Dense)              (None, 5)                 80        
Total params: 265
Trainable params: 265
Non-trainable params: 0
_________________________________________________________________
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


## 三、函数模型
函数式模型是一种创建模型的方法，该模型比tf.keras.Sequential更灵活。函数式模型可以处理具有非线性拓扑的模型，具有共享层的模型以及具有多个输入或输出的模型等等。  
函数式模型存在以下优势：  
（1）易于搭建多输入多输出模型；  
（2）具有共享图层的模型；  
（3）具有非顺序数据流的模型。  
函数式模型通过Model类来实现的。  

### 4.1 模型搭建

In [4]:
#导入相关的包
import tensorboard
import tensorflow as tf
from tensorflow.keras import Sequential, layers, losses, optimizers, datasets

In [5]:
# 输入层
inputs = tf.keras.Input(shape=(1,), name="inputs")
# 隐藏层-1
layer1 = layers.Dense(10, activation="relu", name="layer1")(inputs)
# 隐藏层-2
layer2 = layers.Dense(15, activation="relu", name="layer2")(layer1)
# 输出层
outputs = layers.Dense(5, activation="softmax", name="outputs")(layer2)
# 实例化
model = tf.keras.Model(inputs=inputs, outputs=outputs)
# 展示网络结构
model.summary()
# 绘制网络流程图
keras.utils.plot_model(model, "line-fit-model.png", show_shapes=True)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 1)]               0         
_________________________________________________________________
layer1 (Dense)               (None, 10)                20        
_________________________________________________________________
layer2 (Dense)               (None, 15)                165       
_________________________________________________________________
outputs (Dense)              (None, 5)                 80        
Total params: 265
Trainable params: 265
Non-trainable params: 0
_________________________________________________________________
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


## 四、子类模型

通过子类化tf.keras.Model和定义自己的前向传播模型来构建完全可定制的模型。   
和eager execution模式相辅相成。

In [6]:
class  MyModel(tf.keras.Model):
    """类继承方式搭建神经网络
    参数:
        tf.keras.Model: Model父类
    返回:
        无
    """
    def __init__(self):
        # 继承
        super(MyModel, self).__init__()
        # 隐藏层-1
        self.layer1 = layers.Dense(10, activation=tf.nn.relu, name="layer1")
        # 隐藏层-2
        self.layer2 = layers.Dense(15, activation=tf.nn.relu, name="layer2")
        # 输出层
        self.outputs = layers.Dense(5, activation=tf.nn.softmax, name="outputs")
    def call(self, inputs):
        """实例回调接口，类似重载()
        参数:
            self: 对象
            inputs: 输入数据
        返回:
            输出层张量
        """
        layer1 = self.layer1(inputs)
        layer2 = self.layer2(layer1)
        return self.outputs(layer2)

In [7]:
#对子类模型的使用
inputs = tf.constant([[1]])
print("inputs shape:{}".format(inputs.shape))
model = MyModel()
model(inputs)
model.summary()

inputs shape:(1, 1)
Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               multiple                  20        
_________________________________________________________________
layer2 (Dense)               multiple                  165       
_________________________________________________________________
outputs (Dense)              multiple                  80        
Total params: 265
Trainable params: 265
Non-trainable params: 0
_________________________________________________________________


## 五、总结

对于三种搭建网络的方法，建议初学者重点掌握子类模型。在此对三种搭建网络的方法进行对比。

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161598611720561691615986116125.png"/>